In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
import numpy
import time

from numpy import array
from numpy import hstack
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras import optimizers
from keras.layers import Dropout
from sklearn.metrics import r2_score

import time

from sklearn.metrics import mean_squared_error
from math import sqrt


# fix random seed for reproducibility
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

Using TensorFlow backend.


In [2]:
repeats=5

Data engineering

In [3]:
raw_data=pd.read_csv('Beauty.csv', engine='python', skipfooter=1)

data_train=raw_data.drop('year', axis = 1)
data_train=data_train.drop('month', axis = 1)

# fill missing value with -999
data_train=data_train.fillna(-999)

# labelEncoder 'place_id' and 'LineCategory'
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

categorical=['place_id', 'LineCategory']
print(categorical)

for value in categorical:
    le.fit(data_train[value].astype(str))
    data_train[value]=le.transform(data_train[value].astype(str))
    

    
# Scaler numerical features
numerical = list(data_train.select_dtypes(include=['int64']).columns.values)+list(data_train.select_dtypes(include=['float64']).columns.values)
numerical.append('place_id')
numerical.append('LineCategory')
print(numerical)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler_target = MinMaxScaler()

scaler_target.fit_transform(data_train['Total Product Sale'].values.reshape(-1, 1))
data_train[numerical] = scaler.fit_transform(data_train[numerical])

# one-hot other categorical features
data_train = pd.get_dummies(data_train)

# prepare data for LSTM
dataset=np.array(data_train)
target=dataset[:,[0,1,2]]
Features = []
Targets = []
InputSteps=10
OutputSteps=3
for index in range(len(dataset) - InputSteps): # maxmimum date = lastest date - sequence length
    Features.append(dataset[index: index + InputSteps]) # index : index + steps

for index in range(len(target) - InputSteps-OutputSteps): # maxmimum date = lastest date - sequence length
            Targets.append(target[index+InputSteps: index +InputSteps+OutputSteps]) # index : index + steps
        
Features=np.array(Features)
Targets=np.array(Targets)

Label=0
Features_New=[]
Targets_New=[]
for i in range(len(Targets)):
    Label=0
    for j in range(1,InputSteps):
        if not(Features[i,0,0]==Features[i,j,0] and Features[i,0,1]==Features[i,j,1]):
            Label=1
            #print(Features[i])
    for t in range(OutputSteps):
        if not(Features[i,0,0]==Targets[i,t,0] and Features[i,0,1]==Targets[i,t,1]):
            Label=1
           # print(Targets[i])
    
    if Label==0:
        Features_New.append(Features[i])
        Targets_New.append(Targets[i])
        
Features_New=np.array(Features_New)
Targets_New=np.array(Targets_New)

Targets_New=Targets_New[:,:,2]

Targets_New=numpy.reshape(Targets_New, (Targets_New.shape[0], Targets_New.shape[1],1))

trainLen=round(len(Targets_New)*0.8)
trainX=Features_New[:trainLen]
testX=Features_New[trainLen:]
trainY=Targets_New[:trainLen]
testY=Targets_New[trainLen:]

['place_id', 'LineCategory']
['Total Product Sale', 'No_Of_StatusCO', 'No_Of_Invoiced', 'No_Of_WIP', '1st_frequent_OpCtr', '1st_frequent_No_Of_OpCtr', '2st_frequent_OpCtr', '2st_frequent_No_Of_OpCtr', '1st_frequent_No_Of_ItemType', '2st_frequent_No_Of_ItemType', '3st_frequent_No_Of_ItemType', '4st_frequent_No_Of_ItemType', '5st_frequent_No_Of_ItemType', '1st_frequent_No_Of_ActionType', '2st_frequent_No_Of_ActionType', '3st_frequent_No_Of_ActionType', 'DistinctCount_PartNo', '1st_frequent_No_Of_LineType', 'Total Service Sale', 'Total Qty', 'Total Cost', 'Total GrossProfit', '1stLineCateSales', '2stLineCateSales', '3stLineCateSales', '4stLineCateSales', 'place_id', 'LineCategory']


C:\Users\eyan\AppData\Local\Continuum\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Benchmark

In [5]:
start = time.time()
RMSE_total=0
for i in range(repeats):

    # define model
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(trainX.shape[1],trainX.shape[2])))
    model.add(RepeatVector(3))
    model.add(LSTM(50, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(1)))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(trainX, trainY, epochs=30, verbose=0)

    yhat = model.predict(testX, verbose=0)
    RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
    RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
    RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
    RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
    print('RMSE',RMSE_final)
    RMSE_total+=RMSE_final
    
print('Mean of RMSE',RMSE_total/repeats)
print('Minutes spent', (time.time() - start)/60)

RMSE 8955.766732582562
RMSE 9817.943344066907
RMSE 9496.484770134404
RMSE 8820.275721426262
RMSE 9054.380675103037
Mean of RMSE 9228.970248662634
Minutes spent 25.184386944770814


In [6]:
start = time.time()
RMSE_total=0
for i in range(repeats):

    # define model
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(trainX.shape[1],trainX.shape[2])))
    model.add(RepeatVector(3))
    model.add(LSTM(50, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(1)))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(trainX, trainY, epochs=30, verbose=0)

    yhat = model.predict(testX, verbose=0)
    RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
    RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
    RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
    RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
    print('RMSE',RMSE_final)
    RMSE_total+=RMSE_final
    
print('Mean of RMSE',RMSE_total/repeats)
print('Minutes spent', (time.time() - start)/60)

RMSE 9721.458050084184
RMSE 9036.004193464154
RMSE 9697.604463791728
RMSE 9774.51198433968
RMSE 9509.36989505077
Mean of RMSE 9547.789717346104
Minutes spent 30.064014883836112


# 1 Fit train set well on cost fuhction

# 1.1 Tune learning rate

In [4]:
start = time.time()

LearningRates = [0.0001,0.001, 0.01,0.1,1]
for LearningRate in LearningRates:

    RMSE_total=0
    for i in range(repeats):
        # define model
        model = Sequential()
        model.add(LSTM(50, activation='relu', input_shape=(trainX.shape[1],trainX.shape[2])))
        model.add(RepeatVector(3))
        model.add(LSTM(50, activation='relu', return_sequences=True))
        model.add(TimeDistributed(Dense(1)))
        optimizer = optimizers.Adam(lr=LearningRate)
        model.compile(optimizer=optimizer, loss='mse')
        # fit model
        model.fit(trainX, trainY, epochs=30, verbose=0)

        yhat = model.predict(testX, verbose=0)
        RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
        RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
        RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
        RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
        print('RMSE',RMSE_final)
        RMSE_total+=RMSE_final
    print('Learning Rate',LearningRate)
    print('Mean of RMSE',RMSE_total/repeats)
    
print('Minutes spent', (time.time() - start)/60)

RMSE 10132.590704611612
RMSE 10503.421022600922
RMSE 9722.632583203995
RMSE 11873.42111664614
RMSE 10164.10312859578
Learning Rate 0.0001
Mean of RMSE 10479.23371113169
RMSE 10424.27182608274
RMSE 9151.939613256423
RMSE 9593.07782016842
RMSE 9510.762219773516
RMSE 9257.272766899508
Learning Rate 0.001
Mean of RMSE 9587.464849236121
RMSE 9438.900782192295
RMSE 11688.730062749071
RMSE 10277.58025220658
RMSE 11275.633856593506
RMSE 10704.276325557776
Learning Rate 0.01
Mean of RMSE 10677.024255859846
RMSE 9663.4190440268
RMSE 12585.814620285017
RMSE 9249.939007674771
RMSE 9402.72154764886
RMSE 9479.53798392717
Learning Rate 0.1
Mean of RMSE 10076.286440712523


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [5]:
start = time.time()

LearningRates = [0.001,0.003,0.005,0.007, 0.01,0.03,0.05,0.07]
for LearningRate in LearningRates:

    RMSE_total=0
    for i in range(repeats):
        # define model
        model = Sequential()
        model.add(LSTM(50, activation='relu', input_shape=(trainX.shape[1],trainX.shape[2])))
        model.add(RepeatVector(3))
        model.add(LSTM(50, activation='relu', return_sequences=True))
        model.add(TimeDistributed(Dense(1)))
        optimizer = optimizers.Adam(lr=LearningRate)
        model.compile(optimizer=optimizer, loss='mse')
        # fit model
        model.fit(trainX, trainY, epochs=30, verbose=0)

        yhat = model.predict(testX, verbose=0)
        RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
        RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
        RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
        RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
        print('RMSE',RMSE_final)
        RMSE_total+=RMSE_final
    print('Learning Rate',LearningRate)
    print('Mean of RMSE',RMSE_total/repeats)
    
print('Minutes spent', (time.time() - start)/60)

RMSE 9243.308953045487
RMSE 8942.614342444771
RMSE 9428.424072767282
RMSE 8947.18875858138
RMSE 9508.024589253684
Learning Rate 0.001
Mean of RMSE 9213.912143218522
RMSE 9403.166992439901
RMSE 8929.570431956645
RMSE 10770.416862283651
RMSE 10422.091943509766
RMSE 9364.288005710796
Learning Rate 0.003
Mean of RMSE 9777.906847180153
RMSE 9352.878752887287
RMSE 10287.801808062368
RMSE 8851.596350311456
RMSE 9159.3188684037
RMSE 9112.600831992902
Learning Rate 0.005
Mean of RMSE 9352.839322331543
RMSE 9879.352220579422
RMSE 9035.443541995459
RMSE 13292.967888872896
RMSE 9484.525654376981
RMSE 9411.922784744489
Learning Rate 0.007
Mean of RMSE 10220.842418113849
RMSE 8996.79670348446
RMSE 9361.455824528648
RMSE 8814.125997857218
RMSE 9580.947583503834
RMSE 11252.06170552115
Learning Rate 0.01
Mean of RMSE 9601.077562979062
RMSE 9001.557381315988
RMSE 9581.162375900107
RMSE 8668.74302101584
RMSE 10447.328702339018
RMSE 13592.957742015735
Learning Rate 0.03
Mean of RMSE 10258.349844517339
RMS

# Conclusion: learning rate-0.001

# 1.2 Tune "mini-batch size" and "num of hidder units"


In [4]:
start = time.time()

batch_sizes = [32,64,128,256,512,1024]
neurons = 32
for batch_size in batch_sizes:

    RMSE_total=0
    for i in range(repeats):
        # define model
        model = Sequential()
        model.add(LSTM(neurons, activation='relu', input_shape=(trainX.shape[1],trainX.shape[2])))
        model.add(RepeatVector(3))
        model.add(LSTM(neurons, activation='relu', return_sequences=True))
        model.add(TimeDistributed(Dense(1)))
        optimizer = optimizers.Adam(lr=0.001)
        model.compile(optimizer=optimizer, loss='mse')
        # fit model
        model.fit(trainX, trainY, epochs=200,batch_size=batch_size, verbose=0)

        yhat = model.predict(testX, verbose=0)
        RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
        RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
        RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
        RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
        print('RMSE',RMSE_final)
        RMSE_total+=RMSE_final
    print('batch_size',batch_size)
    print('Mean of RMSE',RMSE_total/repeats)
    
print('Minutes spent', (time.time() - start)/60)

RMSE 9783.681589242331
RMSE 9941.977344999186
RMSE 10722.731304949988
RMSE 9415.684394990583
RMSE 11516.291016236164
batch_size 32
Mean of RMSE 10276.07313008365
RMSE 9929.697473252863
RMSE 10191.74393639272
RMSE 13291.306799822773
RMSE 10220.676014969069
RMSE 10185.074817184306
batch_size 64
Mean of RMSE 10763.699808324347
RMSE 10396.326712969905
RMSE 10811.686843224054
RMSE 11768.237427865126
RMSE 10063.609025979365
RMSE 12142.095877409374
batch_size 128
Mean of RMSE 11036.391177489564
RMSE 9523.679931368142
RMSE 9892.845968027987
RMSE 10397.995691849559
RMSE 10596.463996816034
RMSE 11756.243535479902
batch_size 256
Mean of RMSE 10433.445824708326
RMSE 10394.310947661586
RMSE 9669.264280048277
RMSE 11943.879461527044
RMSE 9674.731357499495
RMSE 10269.45180287542
batch_size 512
Mean of RMSE 10390.327569922363
RMSE 9220.150564731352
RMSE 9546.386163583213
RMSE 9829.445057251587
RMSE 13396.828811496422
RMSE 9200.361738730358
batch_size 1024
Mean of RMSE 10238.634467158587
Minutes spent 

In [ ]:
start = time.time()

batch_sizes = [32,64,128,256,512,1024]
neurons = 64
for batch_size in batch_sizes:

    RMSE_total=0
    for i in range(repeats):
        # define model
        model = Sequential()
        model.add(LSTM(neurons, activation='relu', input_shape=(trainX.shape[1],trainX.shape[2])))
        model.add(RepeatVector(3))
        model.add(LSTM(neurons, activation='relu', return_sequences=True))
        model.add(TimeDistributed(Dense(1)))
        optimizer = optimizers.Adam(lr=0.001)
        model.compile(optimizer=optimizer, loss='mse')
        # fit model
        model.fit(trainX, trainY, epochs=200,batch_size=batch_size, verbose=0)

        yhat = model.predict(testX, verbose=0)
        RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
        RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
        RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
        RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
        print('RMSE',RMSE_final)
        RMSE_total+=RMSE_final
    print('batch_size',batch_size)
    print('Mean of RMSE',RMSE_total/repeats)
    
print('Minutes spent', (time.time() - start)/60)

RMSE 11828.228939566316
RMSE 10696.172756150685
RMSE 11009.121673687203
RMSE 11827.77811212007
RMSE 10998.934141319862
batch_size 32
Mean of RMSE 11272.047124568828
RMSE 10440.926397865094
RMSE 11484.978044673959
RMSE 10635.09754498724
RMSE 11783.039275580142
RMSE 10271.34313583434
batch_size 64
Mean of RMSE 10923.076879788156
RMSE 10581.682737945135
RMSE 10154.382671538775
RMSE 10473.774334694357
RMSE 9760.157863980397
RMSE 10456.833401097605
batch_size 128
Mean of RMSE 10285.366201851253
RMSE 9381.105149774767
RMSE 10362.603475116908
RMSE 9816.874886979704
RMSE 10294.377094630825
RMSE 9076.757678042279
batch_size 256
Mean of RMSE 9786.343656908899
RMSE 15275.181458322208
RMSE 11060.032850307869
RMSE 10744.798125018899
RMSE 10845.491553484142
RMSE 9604.836277856703
batch_size 512
Mean of RMSE 11506.068052997964
RMSE 9581.184345171438
RMSE 11159.999355870592
RMSE 11881.73366843545
RMSE 10888.25903934925
RMSE 11126.220708700317
batch_size 1024
Mean of RMSE 10927.47942350541
Minutes spen

In [ ]:
start = time.time()

batch_sizes = [32,64,128,256,512,1024]
neurons = 128
for batch_size in batch_sizes:

    RMSE_total=0
    for i in range(repeats):
        # define model
        model = Sequential()
        model.add(LSTM(neurons, activation='relu', input_shape=(trainX.shape[1],trainX.shape[2])))
        model.add(RepeatVector(3))
        model.add(LSTM(neurons, activation='relu', return_sequences=True))
        model.add(TimeDistributed(Dense(1)))
        optimizer = optimizers.Adam(lr=0.001)
        model.compile(optimizer=optimizer, loss='mse')
        # fit model
        model.fit(trainX, trainY, epochs=200,batch_size=batch_size, verbose=0)

        yhat = model.predict(testX, verbose=0)
        RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
        RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
        RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
        RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
        print('RMSE',RMSE_final)
        RMSE_total+=RMSE_final
    print('batch_size',batch_size)
    print('Mean of RMSE',RMSE_total/repeats)
    
print('Minutes spent', (time.time() - start)/60)

RMSE 9829.98993617369
RMSE 10914.725772146701
RMSE 9716.496626569156
RMSE 10912.019298478064
RMSE 10562.103593319347
batch_size 32
Mean of RMSE 10387.067045337393
RMSE 10006.670068157184
RMSE 10253.375221942542
RMSE 10639.92908836348
RMSE 9724.89604963478
RMSE 10906.119648473923
batch_size 64
Mean of RMSE 10306.198015314381
RMSE 10324.817259196532
RMSE 10306.601676858269
RMSE 10527.966649202484
RMSE 9916.881996978353


In [ ]:
start = time.time()

batch_sizes = [32,64,128,256,512,1024]
neurons = 256
for batch_size in batch_sizes:

    RMSE_total=0
    for i in range(repeats):
        # define model
        model = Sequential()
        model.add(LSTM(neurons, activation='relu', input_shape=(trainX.shape[1],trainX.shape[2])))
        model.add(RepeatVector(3))
        model.add(LSTM(neurons, activation='relu', return_sequences=True))
        model.add(TimeDistributed(Dense(1)))
        optimizer = optimizers.Adam(lr=0.001)
        model.compile(optimizer=optimizer, loss='mse')
        # fit model
        model.fit(trainX, trainY, epochs=200,batch_size=batch_size, verbose=0)

        yhat = model.predict(testX, verbose=0)
        RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
        RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
        RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
        RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
        print('RMSE',RMSE_final)
        RMSE_total+=RMSE_final
    print('batch_size',batch_size)
    print('Mean of RMSE',RMSE_total/repeats)
    
print('Minutes spent', (time.time() - start)/60)

RMSE 9851.995443265667
RMSE 9305.662765772251
RMSE 10908.717812039633
RMSE 10555.063212735704
RMSE 10374.389590965147
batch_size 32
Mean of RMSE 10199.16576495568
RMSE 10341.370390631584
RMSE 10097.836136888931
RMSE 10482.708233384052
RMSE 10910.953339582296
RMSE 10400.805726631143
batch_size 64
Mean of RMSE 10446.734765423602
RMSE 10278.977163622281
RMSE 9506.124411729164
RMSE 9980.626340815526
RMSE 9867.71611993979
RMSE 9812.565226929773
batch_size 128
Mean of RMSE 9889.201852607306
RMSE 10595.831871157521
RMSE 9869.71542923583
RMSE 10117.471280229278
RMSE 9817.723006485016
RMSE 9562.429701428564
batch_size 256
Mean of RMSE 9992.634257707241
RMSE 9353.827426878495
RMSE 10358.332823615756


In [ ]:
start = time.time()

batch_sizes = [32,64,128,256,512,1024]
neurons = 512
for batch_size in batch_sizes:

    RMSE_total=0
    for i in range(repeats):
        # define model
        model = Sequential()
        model.add(LSTM(neurons, activation='relu', input_shape=(trainX.shape[1],trainX.shape[2])))
        model.add(RepeatVector(3))
        model.add(LSTM(neurons, activation='relu', return_sequences=True))
        model.add(TimeDistributed(Dense(1)))
        optimizer = optimizers.Adam(lr=0.001)
        model.compile(optimizer=optimizer, loss='mse')
        # fit model
        model.fit(trainX, trainY, epochs=200,batch_size=batch_size, verbose=0)

        yhat = model.predict(testX, verbose=0)
        RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
        RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
        RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
        RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
        print('RMSE',RMSE_final)
        RMSE_total+=RMSE_final
    print('batch_size',batch_size)
    print('Mean of RMSE',RMSE_total/repeats)
    
print('Minutes spent', (time.time() - start)/60)

RMSE 10072.204378823706


In [ ]:
start = time.time()

batch_sizes = [32,64,128,256,512,1024]
neurons = 768
for batch_size in batch_sizes:

    RMSE_total=0
    for i in range(repeats):
        # define model
        model = Sequential()
        model.add(LSTM(neurons, activation='relu', input_shape=(trainX.shape[1],trainX.shape[2])))
        model.add(RepeatVector(3))
        model.add(LSTM(neurons, activation='relu', return_sequences=True))
        model.add(TimeDistributed(Dense(1)))
        optimizer = optimizers.Adam(lr=0.001)
        model.compile(optimizer=optimizer, loss='mse')
        # fit model
        model.fit(trainX, trainY, epochs=200,batch_size=batch_size, verbose=0)

        yhat = model.predict(testX, verbose=0)
        RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
        RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
        RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
        RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
        print('RMSE',RMSE_final)
        RMSE_total+=RMSE_final
    print('batch_size',batch_size)
    print('Mean of RMSE',RMSE_total/repeats)
    
print('Minutes spent', (time.time() - start)/60)

In [ ]:
start = time.time()

batch_sizes = [32,64,128,256,512,1024]
neurons = 1024
for batch_size in batch_sizes:

    RMSE_total=0
    for i in range(repeats):
        # define model
        model = Sequential()
        model.add(LSTM(neurons, activation='relu', input_shape=(trainX.shape[1],trainX.shape[2])))
        model.add(RepeatVector(3))
        model.add(LSTM(neurons, activation='relu', return_sequences=True))
        model.add(TimeDistributed(Dense(1)))
        optimizer = optimizers.Adam(lr=0.001)
        model.compile(optimizer=optimizer, loss='mse')
        # fit model
        model.fit(trainX, trainY, epochs=200,batch_size=batch_size, verbose=0)

        yhat = model.predict(testX, verbose=0)
        RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
        RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
        RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
        RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
        print('RMSE',RMSE_final)
        RMSE_total+=RMSE_final
    print('batch_size',batch_size)
    print('Mean of RMSE',RMSE_total/repeats)
    
print('Minutes spent', (time.time() - start)/60)

# Conclusion: 50 neurons, defaut batch size ,30 epochs






# 1.3.1 tune num of hidden layers-have same no. of hidden units in every layer(usually the more the better)

In [6]:
neurons=50

In [10]:
start = time.time()

RMSE_total=0
for i in range(repeats):
    # define model
    model = Sequential()
    model.add(LSTM(neurons, activation='relu', input_shape=(trainX.shape[1],trainX.shape[2])))
    model.add(RepeatVector(3))
    model.add(LSTM(neurons, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(1)))
    optimizer = optimizers.Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    # fit model
    model.fit(trainX, trainY, epochs=30, verbose=0)

    yhat = model.predict(testX, verbose=0)
    RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
    RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
    RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
    RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
    print('RMSE',RMSE_final)
    RMSE_total+=RMSE_final
print('Layers',1)
print('Mean of RMSE',RMSE_total/repeats)

print('Minutes spent', (time.time() - start)/60)

RMSE 8761.039649402788
RMSE 9228.482170624904
RMSE 9274.808146726742
RMSE 9345.47153080255
RMSE 9686.094368912234
Layers 1
Mean of RMSE 9259.179173293844
Minutes spent 88.99117235342662


In [5]:
start = time.time()

RMSE_total=0
for i in range(repeats):
    # define model
    model = Sequential()
    model.add(LSTM(neurons, activation='relu', return_sequences=True, input_shape=(trainX.shape[1],trainX.shape[2])))
    model.add(LSTM(neurons, activation='relu'))
    model.add(RepeatVector(3))
    model.add(LSTM(neurons, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(1)))
    optimizer = optimizers.Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    # fit model
    model.fit(trainX, trainY, epochs=30, verbose=0)

    yhat = model.predict(testX, verbose=0)
    RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
    RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
    RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
    RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
    print('RMSE',RMSE_final)
    RMSE_total+=RMSE_final
print('Layers',2)
print('Mean of RMSE',RMSE_total/repeats)

print('Minutes spent', (time.time() - start)/60)

RMSE 8840.999043712838
RMSE 10042.983391359581
RMSE 9888.904239023757
RMSE 9499.866640070073
RMSE 9529.16685853069
Layers 2
Mean of RMSE 9560.384034539387
Minutes spent 30.73367512623469


In [7]:
start = time.time()

RMSE_total=0
for i in range(repeats):
    # define model
    model = Sequential()
    model.add(LSTM(neurons, activation='relu', return_sequences=True, input_shape=(trainX.shape[1],trainX.shape[2])))
    model.add(LSTM(neurons, activation='relu', return_sequences=True))
    model.add(LSTM(neurons, activation='relu'))
    model.add(RepeatVector(3))
    model.add(LSTM(neurons, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(1)))
    optimizer = optimizers.Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    # fit model
    model.fit(trainX, trainY, epochs=30, verbose=0)

    yhat = model.predict(testX, verbose=0)
    RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
    RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
    RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
    RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
    print('RMSE',RMSE_final)
    RMSE_total+=RMSE_final
print('Layers',3)
print('Mean of RMSE',RMSE_total/repeats)

print('Minutes spent', (time.time() - start)/60)

RMSE 9137.104590092711
RMSE 8680.9866074898
RMSE 9015.842790505705
RMSE 9023.02138076393
RMSE 8968.711048332516
Layers 3
Mean of RMSE 8965.133283436933
Minutes spent 34.76443761587143


In [8]:
start = time.time()

RMSE_total=0
for i in range(repeats):
    # define model
    model = Sequential()
    model.add(LSTM(neurons, activation='relu', return_sequences=True, input_shape=(trainX.shape[1],trainX.shape[2])))
    model.add(LSTM(neurons, activation='relu', return_sequences=True))
    model.add(LSTM(neurons, activation='relu', return_sequences=True))
    model.add(LSTM(neurons, activation='relu'))
    model.add(RepeatVector(3))
    model.add(LSTM(neurons, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(1)))
    optimizer = optimizers.Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    # fit model
    model.fit(trainX, trainY, epochs=30, verbose=0)

    yhat = model.predict(testX, verbose=0)
    RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
    RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
    RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
    RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
    print('RMSE',RMSE_final)
    RMSE_total+=RMSE_final
print('Layers',4)
print('Mean of RMSE',RMSE_total/repeats)

print('Minutes spent', (time.time() - start)/60)

RMSE 9363.606098059878
RMSE 10413.825762012308
RMSE 9822.30502372056
RMSE 9398.648580377778
RMSE 9633.153329826491
Layers 4
Mean of RMSE 9726.307758799403
Minutes spent 50.859784909089406


In [9]:
start = time.time()

RMSE_total=0
for i in range(repeats):
    # define model
    model = Sequential()
    model.add(LSTM(neurons, activation='relu', return_sequences=True, input_shape=(trainX.shape[1],trainX.shape[2])))
    model.add(LSTM(neurons, activation='relu', return_sequences=True))
    model.add(LSTM(neurons, activation='relu', return_sequences=True))
    model.add(LSTM(neurons, activation='relu', return_sequences=True))
    model.add(LSTM(neurons, activation='relu'))
    model.add(RepeatVector(3))
    model.add(LSTM(neurons, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(1)))
    optimizer = optimizers.Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    # fit model
    model.fit(trainX, trainY, epochs=30, verbose=0)

    yhat = model.predict(testX, verbose=0)
    RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
    RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
    RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
    RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
    print('RMSE',RMSE_final)
    RMSE_total+=RMSE_final
print('Layers',5)
print('Mean of RMSE',RMSE_total/repeats)

print('Minutes spent', (time.time() - start)/60)

RMSE 11665.231165603109
RMSE 8838.098316424774
RMSE 9201.4278835823
RMSE 9365.436709477852
RMSE 10220.796064481023
Layers 5
Mean of RMSE 9858.198027913812
Minutes spent 80.75290323098501


# Conclusion:3 layer


# 1.3.2 tune learning rate decay

In [ ]:
start = time.time()

decays = [0,0.3,0.5,0.7,1.0,1.3,1.5]

for decay in decays:

    RMSE_total=0
    for i in range(repeats):
        model = Sequential()
        model.add(LSTM(neurons, activation='relu', return_sequences=True, input_shape=(trainX.shape[1],trainX.shape[2])))
        model.add(LSTM(neurons, activation='relu', return_sequences=True))
        model.add(LSTM(neurons, activation='relu'))
        model.add(RepeatVector(3))
        model.add(LSTM(neurons, activation='relu', return_sequences=True))
        model.add(TimeDistributed(Dense(1)))
        optimizer = optimizers.Adam(decay=decay,lr=0.001)
        model.compile(optimizer=optimizer, loss='mse')
        # fit model
        model.fit(trainX, trainY, epochs=30, verbose=0)

        yhat = model.predict(testX, verbose=0)
        RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
        RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
        RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
        RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
        print('RMSE',RMSE_final)
        RMSE_total+=RMSE_final
    print('decay',decay)
    print('Mean of RMSE',RMSE_total/repeats)
    
print('Minutes spent', (time.time() - start)/60)

RMSE 8808.171808131001
RMSE 8872.074060385165
RMSE 8723.43800371222
RMSE 9155.679608547654
RMSE 8936.064690696725
decay 0
Mean of RMSE 8899.085634294552
RMSE 9272.458635584831
RMSE 9297.87851538039
RMSE 9050.942919378962
RMSE 9394.505872241163
RMSE 9349.973182322408
decay 0.3
Mean of RMSE 9273.15182498155
RMSE 9518.236861398063
RMSE 9224.862916080745
RMSE 9936.400081643136
RMSE 9342.675826893581
RMSE 9380.866820090754
decay 0.5
Mean of RMSE 9480.608501221257
RMSE 9362.975801142911
RMSE 9490.907338737556
RMSE 9618.466644890928
RMSE 9223.784074950901
RMSE 9543.060334109126
decay 0.7
Mean of RMSE 9447.838838766284
RMSE 8999.802851724216
RMSE 9373.347807889542
RMSE 9285.34224178081
RMSE 9154.950522862815
RMSE 9126.902738879573
decay 1.0
Mean of RMSE 9188.069232627391
RMSE 9178.027125359713
RMSE 9032.974462075166
RMSE 9320.808261083997


# 1.4 tune activation and weights initialization(he+relu;tanh+Xavier)

In [7]:
neurons=50

In [5]:
start = time.time()


init_modes = ['uniform',  'normal', 'zero', 'glorot_uniform', 'he_normal', 'he_uniform']
activation ='relu'

start = time.time()
for init_mode in init_modes:

    RMSE_total=0
    for i in range(repeats):
        # define model
        model = Sequential()
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation,return_sequences=True, input_shape=(trainX.shape[1],trainX.shape[2])))
        model.add(LSTM(neurons, kernel_initializer=init_mode, activation=activation, return_sequences=True))
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation))
        model.add(RepeatVector(3))
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation, return_sequences=True))
        model.add(TimeDistributed(Dense(1)))
        optimizer = optimizers.Adam(decay=0,lr=0.001)
        model.compile(optimizer=optimizer, loss='mse')
        # fit model
        model.fit(trainX, trainY, epochs=30, verbose=0)

        yhat = model.predict(testX, verbose=0)
        RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
        RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
        RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
        RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
        print('RMSE',RMSE_final)
        RMSE_total+=RMSE_final
    print('activation',activation,'init_mode',init_mode)
    print('Mean of RMSE',RMSE_total/repeats)
    
print('Minutes spent', (time.time() - start)/60)

RMSE 8809.143285717306
RMSE 8681.741996213248
RMSE 9373.262986698714
RMSE 9264.314680005979
RMSE 9676.064970104415
activation relu init_mode uniform
Mean of RMSE 9160.905583747932
RMSE 8723.92726195268
RMSE 8849.977422419548
RMSE 8889.846214671767
RMSE 8824.04234756667
RMSE 9050.984614524974
activation relu init_mode normal
Mean of RMSE 8867.755572227128
RMSE 9019.498561085327
RMSE 9316.221802491013
RMSE 9923.375440384247
RMSE 9408.197717834873
RMSE 9137.336599075696
activation relu init_mode zero
Mean of RMSE 9360.92602417423
RMSE 8964.077952176924
RMSE 9753.217149034774
RMSE 9286.848487306532
RMSE 9248.177034353075
RMSE 8725.442352846787
activation relu init_mode glorot_uniform
Mean of RMSE 9195.552595143618
RMSE 8766.655563279635
RMSE 8915.326937812031
RMSE 9511.62392674853
RMSE 9252.286085916145
RMSE 9453.58716461466
activation relu init_mode he_normal
Mean of RMSE 9179.895935674202
RMSE 9309.248221068518
RMSE 11113.454801436694
RMSE 9832.12034748002
RMSE 8796.02500436404
RMSE 8817

In [6]:
init_modes = ['uniform',  'normal', 'zero', 'glorot_uniform', 'he_normal', 'he_uniform']
activation ='softmax'

start = time.time()
for init_mode in init_modes:

    RMSE_total=0
    for i in range(repeats):
        # define model
        model = Sequential()
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation,return_sequences=True, input_shape=(trainX.shape[1],trainX.shape[2])))
        model.add(LSTM(neurons, kernel_initializer=init_mode, activation=activation, return_sequences=True))
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation))
        model.add(RepeatVector(3))
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation, return_sequences=True))
        model.add(TimeDistributed(Dense(1)))
        optimizer = optimizers.Adam(decay=0,lr=0.001)
        model.compile(optimizer=optimizer, loss='mse')
        # fit model
        model.fit(trainX, trainY, epochs=30, verbose=0)

        yhat = model.predict(testX, verbose=0)
        RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
        RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
        RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
        RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
        print('RMSE',RMSE_final)
        RMSE_total+=RMSE_final
    print('activation',activation,'init_mode',init_mode)
    print('Mean of RMSE',RMSE_total/repeats)
    
print('Minutes spent', (time.time() - start)/60)

RMSE 9920.11046217443
RMSE 9071.962543625172
RMSE 9863.831517323057
RMSE 8860.467336916603
RMSE 8942.554703053096
activation softmax init_mode uniform
Mean of RMSE 9331.785312618473
RMSE 9848.29151955747
RMSE 9914.5612269324
RMSE 9339.280881677028
RMSE 9112.44471057891
RMSE 9551.974338755277
activation softmax init_mode normal
Mean of RMSE 9553.310535500215
RMSE 8865.217912663307
RMSE 11044.285941294795
RMSE 9825.94237948299
RMSE 10770.716081392413
RMSE 9890.38733310788
activation softmax init_mode zero
Mean of RMSE 10079.309929588277
RMSE 8965.061095121162
RMSE 8845.983965500183
RMSE 10181.769153418429
RMSE 9190.497757506113
RMSE 9582.763636981757
activation softmax init_mode glorot_uniform
Mean of RMSE 9353.215121705529
RMSE 9468.383384548093
RMSE 9939.907519611756
RMSE 9934.292941834126
RMSE 9745.142148200748
RMSE 9525.546641623621
activation softmax init_mode he_normal
Mean of RMSE 9722.654527163668
RMSE 9463.519250489016
RMSE 10071.624874914902
RMSE 9750.593304390357
RMSE 10135.31

In [7]:
init_modes = ['uniform',  'normal', 'zero', 'glorot_uniform', 'he_normal', 'he_uniform']
activation ='softplus'

start = time.time()
for init_mode in init_modes:

    RMSE_total=0
    for i in range(repeats):
        # define model
        model = Sequential()
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation,return_sequences=True, input_shape=(trainX.shape[1],trainX.shape[2])))
        model.add(LSTM(neurons, kernel_initializer=init_mode, activation=activation, return_sequences=True))
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation))
        model.add(RepeatVector(3))
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation, return_sequences=True))
        model.add(TimeDistributed(Dense(1)))
        optimizer = optimizers.Adam(decay=0,lr=0.001)
        model.compile(optimizer=optimizer, loss='mse')
        # fit model
        model.fit(trainX, trainY, epochs=30, verbose=0)

        yhat = model.predict(testX, verbose=0)
        RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
        RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
        RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
        RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
        print('RMSE',RMSE_final)
        RMSE_total+=RMSE_final
    print('activation',activation,'init_mode',init_mode)
    print('Mean of RMSE',RMSE_total/repeats)
    
print('Minutes spent', (time.time() - start)/60)

RMSE 9805.300008593937
RMSE 8821.03164676623
RMSE 10254.98033322049
RMSE 9267.685128386285
RMSE 10857.57809544548
activation softplus init_mode uniform
Mean of RMSE 9801.315042482485
RMSE 10144.659764828444
RMSE 10080.369012717298
RMSE 9204.897242250796
RMSE 8771.654157869905
RMSE 9189.109626759348
activation softplus init_mode normal
Mean of RMSE 9478.137960885157
RMSE 9411.668544840046
RMSE 9049.31785162634
RMSE 11458.619295247574
RMSE 8942.194457611091
RMSE 11464.791236870362
activation softplus init_mode zero
Mean of RMSE 10065.318277239083
RMSE 9840.637066233305
RMSE 9521.234414560815
RMSE 13057.402131451834
RMSE 10317.78701325087
RMSE 8955.101829685322
activation softplus init_mode glorot_uniform
Mean of RMSE 10338.432491036428
RMSE 14389.390651976011
RMSE 12856.83473270661
RMSE 9951.493707372354
RMSE 9667.006108814183
RMSE 11524.673458466203
activation softplus init_mode he_normal
Mean of RMSE 11677.879731867073
RMSE 11688.032798373024
RMSE 8817.635135961651
RMSE 10099.924345179

In [8]:
init_modes = ['uniform',  'normal', 'zero', 'glorot_uniform', 'he_normal', 'he_uniform']
activation ='tanh'

start = time.time()
for init_mode in init_modes:

    RMSE_total=0
    for i in range(repeats):
        # define model
        model = Sequential()
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation,return_sequences=True, input_shape=(trainX.shape[1],trainX.shape[2])))
        model.add(LSTM(neurons, kernel_initializer=init_mode, activation=activation, return_sequences=True))
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation))
        model.add(RepeatVector(3))
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation, return_sequences=True))
        model.add(TimeDistributed(Dense(1)))
        optimizer = optimizers.Adam(decay=0,lr=0.001)
        model.compile(optimizer=optimizer, loss='mse')
        # fit model
        model.fit(trainX, trainY, epochs=30, verbose=0)

        yhat = model.predict(testX, verbose=0)
        RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
        RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
        RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
        RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
        print('RMSE',RMSE_final)
        RMSE_total+=RMSE_final
    print('activation',activation,'init_mode',init_mode)
    print('Mean of RMSE',RMSE_total/repeats)
    
print('Minutes spent', (time.time() - start)/60)

RMSE 9338.725136398121
RMSE 8886.985730465569
RMSE 9248.20706848716
RMSE 9288.021551467638
RMSE 9055.044388392274
activation tanh init_mode uniform
Mean of RMSE 9163.396775042153
RMSE 8895.383392801916
RMSE 8901.364388604996
RMSE 9071.69997100187
RMSE 9364.81036789957
RMSE 10195.504997880838
activation tanh init_mode normal
Mean of RMSE 9285.752623637838
RMSE 9019.334494843859
RMSE 9317.410177903748
RMSE 9899.769131330493
RMSE 9407.123744779125
RMSE 9662.333465541533
activation tanh init_mode zero
Mean of RMSE 9461.194202879751
RMSE 8937.07611109498
RMSE 8907.390148602526
RMSE 8898.641573726125
RMSE 9126.740937811595
RMSE 9271.835787392458
activation tanh init_mode glorot_uniform
Mean of RMSE 9028.336911725535
RMSE 9279.59989665024
RMSE 8994.119490261493
RMSE 9595.098002970595
RMSE 8878.772842844486
RMSE 8945.0848994339
activation tanh init_mode he_normal
Mean of RMSE 9138.535026432142
RMSE 9069.663276439553
RMSE 9178.178826270523
RMSE 9006.536964362793
RMSE 9025.818847751905
RMSE 9283

In [ ]:
init_modes = ['uniform',  'normal', 'zero', 'glorot_uniform', 'he_normal', 'he_uniform']
activation ='sigmoid'

start = time.time()
for init_mode in init_modes:

    RMSE_total=0
    for i in range(repeats):
        # define model
        model = Sequential()
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation,return_sequences=True, input_shape=(trainX.shape[1],trainX.shape[2])))
        model.add(LSTM(neurons, kernel_initializer=init_mode, activation=activation, return_sequences=True))
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation))
        model.add(RepeatVector(3))
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation, return_sequences=True))
        model.add(TimeDistributed(Dense(1)))
        optimizer = optimizers.Adam(decay=0,lr=0.001)
        model.compile(optimizer=optimizer, loss='mse')
        # fit model
        model.fit(trainX, trainY, epochs=30, verbose=0)

        yhat = model.predict(testX, verbose=0)
        RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
        RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
        RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
        RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
        print('RMSE',RMSE_final)
        RMSE_total+=RMSE_final
    print('activation',activation,'init_mode',init_mode)
    print('Mean of RMSE',RMSE_total/repeats)
    
print('Minutes spent', (time.time() - start)/60)

RMSE 10316.734782600715
RMSE 11649.205291757093
RMSE 9456.656973184006
RMSE 10412.28019050884
RMSE 8788.165577325855
activation sigmoid init_mode uniform
Mean of RMSE 10124.608563075302
RMSE 11153.919260276443
RMSE 9157.827620301663
RMSE 10031.150577041679
RMSE 10180.787426270907
RMSE 9964.691171355635
activation sigmoid init_mode normal
Mean of RMSE 10097.675211049265
RMSE 10515.548337513448
RMSE 9239.477473092566
RMSE 11823.365746572954
RMSE 9795.671803821846
RMSE 9939.76584663697
activation sigmoid init_mode zero
Mean of RMSE 10262.765841527556
RMSE 8860.042855375885
RMSE 8832.672006575158
RMSE 8695.133652589195
RMSE 10082.958636444871
RMSE 10236.58799689021
activation sigmoid init_mode glorot_uniform
Mean of RMSE 9341.479029575063
RMSE 9487.488832988203
RMSE 9340.559501787639
RMSE 9209.678555213684
RMSE 9158.682321179618
RMSE 9451.826378043026
activation sigmoid init_mode he_normal
Mean of RMSE 9329.647117842434
RMSE 10287.079116766088
RMSE 11104.418388182383
RMSE 9092.486735536646

In [ ]:
init_modes = ['uniform',  'normal', 'zero', 'glorot_uniform', 'he_normal', 'he_uniform']
activation ='linear'

start = time.time()
for init_mode in init_modes:

    RMSE_total=0
    for i in range(repeats):
        # define model
        model = Sequential()
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation,return_sequences=True, input_shape=(trainX.shape[1],trainX.shape[2])))
        model.add(LSTM(neurons, kernel_initializer=init_mode, activation=activation, return_sequences=True))
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation))
        model.add(RepeatVector(3))
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation, return_sequences=True))
        model.add(TimeDistributed(Dense(1)))
        optimizer = optimizers.Adam(decay=0,lr=0.001)
        model.compile(optimizer=optimizer, loss='mse')
        # fit model
        model.fit(trainX, trainY, epochs=30, verbose=0)

        yhat = model.predict(testX, verbose=0)
        RMSE_1=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,0,:]),scaler_target.inverse_transform(yhat[:,0,:])))
        RMSE_2=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,1,:]),scaler_target.inverse_transform(yhat[:,1,:])))                                 
        RMSE_3=sqrt(mean_squared_error(scaler_target.inverse_transform(testY[:,2,:]),scaler_target.inverse_transform(yhat[:,2,:])))                                    
        RMSE_final=(RMSE_1+RMSE_2+RMSE_3)/3
        print('RMSE',RMSE_final)
        RMSE_total+=RMSE_final
    print('activation',activation,'init_mode',init_mode)
    print('Mean of RMSE',RMSE_total/repeats)
    
print('Minutes spent', (time.time() - start)/60)

# 1.5 tune Batch Normalization(use it or not) ??


In [ ]:
# import BatchNormalization
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation, Dense

def create_model():
	# create model
    model = Sequential()
    # layer_1
    model.add(Dense(32, input_dim=640,kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.0))
    # layer_2
    model.add(Dense(32,kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.0))
    # layer_3
    model.add(Dense(1,kernel_initializer='normal'))
	# Compile model
    optimizer = optimizers.Adagrad(decay=0.0) 
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])
    return model

# create model
model = KerasRegressor(build_fn=create_model, epochs=500, batch_size=256, verbose=0)



model.fit(X_train, y_train)



y_predict=model.predict(X_test)

from math import sqrt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as mae

print('r2_score:')
print(r2_score(y_test,y_predict))

y_test_origi=scaler_target.inverse_transform(y_test)
y_predict=y_predict.reshape(-1, 1)
y_predict_origi=scaler_target.inverse_transform(y_predict)

print('MAE:')
print(mae(y_test_origi, y_predict_origi))


# 2 Fit dev set well on cost fuhction

# 2.1 tune Dropout Regularization

In [ ]:
dropout_rates = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
weight_constraint =1 

for dropout_rate in dropout_rates:

    R2Score_total=0
    for i in range(repeats):
        model = Sequential()
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation,kernel_constraint=max_norm(weight_constraint), input_shape=(trainX.shape[1],trainX.shape[2])))
         model.add(Dropout(dropout_rate))
        model.add(RepeatVector(3))
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation,return_sequences=True))
        optimizer = optimizers.Adam(decay=decay)
        model.compile(optimizer=optimizer, loss='mse')
        # fit model
        model.fit(trainX, trainY, epochs=100,batch_size=batch_size, verbose=0)

        yhat = model.predict(testX, verbose=0)
        print((r2_score(testY[:,0,:],yhat[:,0,:])+r2_score(testY[:,1,:],yhat[:,1,:])+r2_score(testY[:,2,:],yhat[:,2,:]))/3)
        R2Score_total+=(r2_score(testY[:,0,:],yhat[:,0,:])+r2_score(testY[:,1,:],yhat[:,1,:])+r2_score(testY[:,2,:],yhat[:,2,:]))/3
    print('dropout_rate',dropout_rate)
    print('Mean of R2 score',R2Score_total/repeats)
    
print('Minutes spent', (time.time() - start)/60)

In [ ]:
dropout_rates = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
weight_constraint =2 

for dropout_rate in dropout_rates:

    R2Score_total=0
    for i in range(repeats):
        model = Sequential()
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation,kernel_constraint=max_norm(weight_constraint), input_shape=(trainX.shape[1],trainX.shape[2])))
         model.add(Dropout(dropout_rate))
        model.add(RepeatVector(3))
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation,return_sequences=True))
        optimizer = optimizers.Adam(decay=decay)
        model.compile(optimizer=optimizer, loss='mse')
        # fit model
        model.fit(trainX, trainY, epochs=100,batch_size=batch_size, verbose=0)

        yhat = model.predict(testX, verbose=0)
        print((r2_score(testY[:,0,:],yhat[:,0,:])+r2_score(testY[:,1,:],yhat[:,1,:])+r2_score(testY[:,2,:],yhat[:,2,:]))/3)
        R2Score_total+=(r2_score(testY[:,0,:],yhat[:,0,:])+r2_score(testY[:,1,:],yhat[:,1,:])+r2_score(testY[:,2,:],yhat[:,2,:]))/3
    print('dropout_rate',dropout_rate)
    print('Mean of R2 score',R2Score_total/repeats)
    
print('Minutes spent', (time.time() - start)/60)

In [ ]:
dropout_rates = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
weight_constraint =3 

for dropout_rate in dropout_rates:

    R2Score_total=0
    for i in range(repeats):
        model = Sequential()
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation,kernel_constraint=max_norm(weight_constraint), input_shape=(trainX.shape[1],trainX.shape[2])))
         model.add(Dropout(dropout_rate))
        model.add(RepeatVector(3))
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation,return_sequences=True))
        optimizer = optimizers.Adam(decay=decay)
        model.compile(optimizer=optimizer, loss='mse')
        # fit model
        model.fit(trainX, trainY, epochs=100,batch_size=batch_size, verbose=0)

        yhat = model.predict(testX, verbose=0)
        print((r2_score(testY[:,0,:],yhat[:,0,:])+r2_score(testY[:,1,:],yhat[:,1,:])+r2_score(testY[:,2,:],yhat[:,2,:]))/3)
        R2Score_total+=(r2_score(testY[:,0,:],yhat[:,0,:])+r2_score(testY[:,1,:],yhat[:,1,:])+r2_score(testY[:,2,:],yhat[:,2,:]))/3
    print('dropout_rate',dropout_rate)
    print('Mean of R2 score',R2Score_total/repeats)
    
print('Minutes spent', (time.time() - start)/60)

In [ ]:
dropout_rates = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
weight_constraint =4 

for dropout_rate in dropout_rates:

    R2Score_total=0
    for i in range(repeats):
        model = Sequential()
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation,kernel_constraint=max_norm(weight_constraint), input_shape=(trainX.shape[1],trainX.shape[2])))
         model.add(Dropout(dropout_rate))
        model.add(RepeatVector(3))
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation,return_sequences=True))
        optimizer = optimizers.Adam(decay=decay)
        model.compile(optimizer=optimizer, loss='mse')
        # fit model
        model.fit(trainX, trainY, epochs=100,batch_size=batch_size, verbose=0)

        yhat = model.predict(testX, verbose=0)
        print((r2_score(testY[:,0,:],yhat[:,0,:])+r2_score(testY[:,1,:],yhat[:,1,:])+r2_score(testY[:,2,:],yhat[:,2,:]))/3)
        R2Score_total+=(r2_score(testY[:,0,:],yhat[:,0,:])+r2_score(testY[:,1,:],yhat[:,1,:])+r2_score(testY[:,2,:],yhat[:,2,:]))/3
    print('dropout_rate',dropout_rate)
    print('Mean of R2 score',R2Score_total/repeats)
    
print('Minutes spent', (time.time() - start)/60)

In [ ]:
dropout_rates = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
weight_constraint =5

for dropout_rate in dropout_rates:

    R2Score_total=0
    for i in range(repeats):
        model = Sequential()
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation,kernel_constraint=max_norm(weight_constraint), input_shape=(trainX.shape[1],trainX.shape[2])))
         model.add(Dropout(dropout_rate))
        model.add(RepeatVector(3))
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation,return_sequences=True))
        optimizer = optimizers.Adam(decay=decay)
        model.compile(optimizer=optimizer, loss='mse')
        # fit model
        model.fit(trainX, trainY, epochs=100,batch_size=batch_size, verbose=0)

        yhat = model.predict(testX, verbose=0)
        print((r2_score(testY[:,0,:],yhat[:,0,:])+r2_score(testY[:,1,:],yhat[:,1,:])+r2_score(testY[:,2,:],yhat[:,2,:]))/3)
        R2Score_total+=(r2_score(testY[:,0,:],yhat[:,0,:])+r2_score(testY[:,1,:],yhat[:,1,:])+r2_score(testY[:,2,:],yhat[:,2,:]))/3
    print('dropout_rate',dropout_rate)
    print('Mean of R2 score',R2Score_total/repeats)
    
print('Minutes spent', (time.time() - start)/60)

# 2 tune epoch, and re-tune learning rate

In [ ]:
epochs = [10, 50, 100,300,500,700,1000]
learn_rate = 

for epoch in epochs:

    R2Score_total=0
    for i in range(repeats):
        model = Sequential()
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation,kernel_constraint=max_norm(weight_constraint), input_shape=(trainX.shape[1],trainX.shape[2])))
         model.add(Dropout(dropout_rate))
        model.add(RepeatVector(3))
        model.add(LSTM(neurons,kernel_initializer=init_mode, activation=activation,return_sequences=True))
        optimizer = optimizers.Adam(lr=learn_rate)
        model.compile(optimizer=optimizer, loss='mse')
        # fit model
        model.fit(trainX, trainY, epochs=epoch,batch_size=batch_size, verbose=0)

        yhat = model.predict(testX, verbose=0)
        print((r2_score(testY[:,0,:],yhat[:,0,:])+r2_score(testY[:,1,:],yhat[:,1,:])+r2_score(testY[:,2,:],yhat[:,2,:]))/3)
        R2Score_total+=(r2_score(testY[:,0,:],yhat[:,0,:])+r2_score(testY[:,1,:],yhat[:,1,:])+r2_score(testY[:,2,:],yhat[:,2,:]))/3
    print('epoch',epoch)
    print('Mean of R2 score',R2Score_total/repeats)
    
print('Minutes spent', (time.time() - start)/60)